In [1]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim

sentences = [
    'The new movie is awesome',
    'This recent movie is so good',
]

model = SentenceTransformer('Supabase/gte-small')
embeddings = model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

c:\ProgramData\Miniconda\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name Supabase/gte-small. Creating a new one with MEAN pooling.
c:\ProgramData\Miniconda\envs\py310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\ProgramData\Miniconda\envs\py310\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kate\.cache\huggingface\hub\models--Supabase--gte-small. Caching files will still work b

tensor([[0.8980]])


In [2]:
# Two lists of sentences
sentences1 = [
    "The cat sits outside",
    "A man is playing guitar",
    "The new movie is awesome",
]

sentences2 = [
    "The dog plays in the garden",
    "A woman watches TV",
    "中文是这个",
]

# Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = cos_sim(embeddings1, embeddings2)

# Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(
        sentences1[i], sentences2[i], cosine_scores[i][i]
    ))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.8200
A man is playing guitar 		 A woman watches TV 		 Score: 0.7016
The new movie is awesome 		 中文是这个 		 Score: 0.7315


In [3]:
# Our sentences to encode
sentences = [
    "This framework generates embeddings for each input sentence",
    "Sentences are passed as a list of string.",
    "The quick brown fox jumps over the lazy dog."
]

# Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

# Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    print("Embedding:", embedding)
    print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-4.56867486e-01 -6.03738651e-02  2.77015064e-02 -1.49324194e-01
 -2.58539319e-02  3.99035335e-01 -1.55862225e-02  2.39103287e-01
  9.95152816e-02  1.49657965e-01 -3.45251858e-01 -4.33489144e-01
  6.84537292e-01  2.49792680e-01  3.92542034e-01  3.05619180e-01
 -2.38010973e-01  3.97296101e-01 -4.60436344e-01 -1.37540281e-01
  5.90817988e-01 -2.84304410e-01  1.05978265e-01 -5.92266262e-01
 -1.59350410e-01  4.13091540e-01 -1.64931953e-01 -7.34146684e-02
 -3.01011443e-01 -1.89854693e+00  2.36650016e-02 -5.51726103e-01
  7.99842477e-01 -4.33838964e-02 -2.60188252e-01 -1.74995959e-01
 -4.91537243e-01  4.09644336e-01 -1.80871010e-01  2.30170995e-01
  2.36194879e-01  2.71462470e-01  2.17981264e-02 -6.09191775e-01
 -2.04824224e-01 -5.56082666e-01 -6.08014166e-01  7.77903624e-05
 -8.24693516e-02 -2.05188289e-01 -7.09772557e-02 -4.21118706e-01
 -9.76334512e-02  8.62646177e-02  2.12224036e-01  1.12527385e-01
  2.59943

In [4]:
from sentence_transformers.util import semantic_search

docs = [
    "A man is eating food.",
    "A man is eating a piece of bread.",
    "The girl is carrying a baby.",
    "A man is riding a horse.",
    "A woman is playing violin.",
    "Two men pushed carts through the woods.",
    "A man is riding a white horse on an enclosed ground.",
    "A monkey is playing drums.",
    "A cheetah is running behind its prey.",
]

docs_embeddings = model.encode(docs, convert_to_tensor=True)

query = "tell me about music"
query_embedding = model.encode(query, convert_to_tensor=True)

hits = semantic_search(query_embedding, docs_embeddings, top_k=2)
hits

for hit in hits[0]:
    print(docs[hit['corpus_id']], "(Score: %.4f)" % hit['score'])

A woman is playing violin. (Score: 0.7764)
A monkey is playing drums. (Score: 0.7570)


In [5]:
import tiktoken

def split_large_text(large_text, max_tokens):
    enc = tiktoken.get_encoding("cl100k_base")
    tokenized_text = enc.encode(large_text)

    chunks = []
    current_chunk = []
    current_length = 0

    for token in tokenized_text:
        current_chunk.append(token)
        current_length += 1

        if current_length >= max_tokens:
            chunks.append(enc.decode(current_chunk).rstrip(' .,;'))
            current_chunk = []
            current_length = 0

    if current_chunk:
        chunks.append(enc.decode(current_chunk).rstrip(' .,;'))

    return chunks

Why use tokens?

> By breaking words into smaller parts (tokens), LLMs can better handle new or unusual words by understanding their building blocks. It also helps the model grasp the nuances of language, such as different word forms and contextual meanings.

[source](https://kelvin.legal/understanding-large-language-models-words-versus-tokens/#:~:text=By%20breaking%20words%20into%20smaller,word%20forms%20and%20contextual%20meanings.)

In [6]:
import tiktoken

sent = "If we split a text by number of characters, it is not obvious how many tokens these chunks will be."


print(len(sent.split()))

enc = tiktoken.get_encoding("cl100k_base")
encoded = enc.encode(sent)

print(len(encoded))
tokens = [enc.decode_single_token_bytes(x) for x in encoded]
print(tokens)
print(len(tokens))


decoded = enc.decode(encoded)
print(len(decoded.split()))
decoded


20
22
[b'If', b' we', b' split', b' a', b' text', b' by', b' number', b' of', b' characters', b',', b' it', b' is', b' not', b' obvious', b' how', b' many', b' tokens', b' these', b' chunks', b' will', b' be', b'.']
22
20


'If we split a text by number of characters, it is not obvious how many tokens these chunks will be.'

In [7]:
import google.generativeai as genai
import time
import os

import getpass

GOOGLE_API_KEY = getpass.getpass()


In [12]:

genai.configure(api_key=GOOGLE_API_KEY)
gen_model = genai.GenerativeModel('gemini-pro')

doc = '''
What Is Bitcoin Halving? Definition, How It Works, Why It Matters
By LUKE CONWAY Updated April 19, 2024
Reviewed by ERIKA RASURE
Fact checked by VIKKI VELASQUEZ
Bitcoin Halving
Investopedia / NoNo Flores

Trending Videos
Close this video player
What Is Bitcoin Halving?
The Bitcoin Halving is when Bitcoin's mining reward is split in half. It takes the blockchain network about four years to open 210,000 more blocks, a standard set by the blockchain's creators to continuously reduce the rate at which the cryptocurrency is introduced.

The first reward was 50 bitcoin. Previous halving dates were:
Nov. 28, 2012, to 25 bitcoins
July 9, 2016, to 12.5 bitcoins
May 11, 2020, to 6.25 bitcoins
April 19, 2024, to 3.125 bitcoins
The next halving is expected to occur in 2028, when the block reward will fall to 1.625 BTC.

As of April 2024, about 19.69 million bitcoins were in circulation, leaving just around 1.31 million to be released via mining rewards.
1

Take the Next Step to Invest
Advertiser Disclosure



KEY TAKEAWAYS
A Bitcoin halving event occurs when the reward for mining Bitcoin transactions is cut in half.
Halvings reduce the rate at which new coins are created and thus lower the available amount of new supply.
Bitcoin last halved on April 19, 2024, resulting in a block reward of 3.125 BTC.
The final halving is expected to occur in 2140, when the number of bitcoins circulating will reach the theoretical maximum supply of 21 million.
Basics of the Bitcoin Network
To understand a Bitcoin halving, you must first know how the Bitcoin network operates.

Bitcoin's underlying technology, blockchain, consists of a network of computers (called nodes) that run Bitcoin's software and contain a partial or complete history of transactions occurring on its network. Each full node contains the entire history of transactions on Bitcoin and is responsible for approving or rejecting a transaction in Bitcoin's network. To do that, the node conducts a check to ensure the transaction is valid. These include ensuring the transaction contains the correct validation parameters and does not exceed the required length.

Each transaction is approved individually. This is said to occur only after all the transactions contained in a block are approved. After approval, the transaction is appended to the existing blockchain and broadcast to other nodes.
2

Adding more computers (or nodes) to the blockchain increases its stability and security. There were 19,329 nodes estimated to be running Bitcoin's code on April 19, 2024.
3
 Although anyone can participate in Bitcoin's network as a node as long as they have enough storage to download the entire blockchain and its history of transactions, not all of them are miners.

Basics of Bitcoin Mining
Bitcoin mining is the process by which people use computers or mining hardware to participate in Bitcoin's blockchain network as transaction processors and validators. Miners receive rewards and transaction fees.

Bitcoin uses a system called proof-of-work (PoW) to validate transaction information. It's called proof-of-work because solving the cryptographic puzzle takes time and energy, which acts as proof that work was done.

The term mining is not used literally but as a reference to how precious metals are harvested. When a block is filled with transactions, it is closed and sent to a mining queue. Once it is queued up for verification, Bitcoin miners compete to be the first to find a number with a value less than that of a target set by the network. The hash is a hexadecimal number that contains all of the encrypted information of the previous blocks.

Mining confirms the legitimacy of the transactions in a block and opens a new one. Nodes then verify the transactions further in a series of confirmations. This process creates a chain of blocks containing information, forming the blockchain.
4

Transaction verification and immutability are the primary intent behind the blockchain network and consensus mechanism. The Bitcoin reward is a byproduct of the mining process that acts as an incentive to participate in securing the blockchain.

Bitcoin Halving Effects
Inflation
One of the key concepts behind halving the reward is to address inflation concerns. Inflation is a decrease in the amount of goods a certain amount of currency can buy at any given moment. In the U.S., inflation is measured by how much it costs to buy a basket of goods. There is an acceptable inflation rate that is considered good for an economy—usually 2%—but this number is generally a target set by central banks as a goal rather than a reachable figure.

The Bitcoin Halving is intended to counter any inflationary effects on Bitcoin by lowering the reward amount and maintaining scarcity. However, this inflation "protection" mechanism does not protect Bitcoin users from the inflationary effects of the fiat currency to which it must be converted to be used in an economy.

Gains made regarding market value might offer inflation protection for investors, but it doesn't for the cryptocurrency's intended use as a payment method.

Demand
Because a halving reduces the number of new Bitcoins introduced, demand for new Bitcoins generally increases. This can be noted by looking at Bitcoin's price after each previous halving event—it has generally risen.

Investing
Bitcoin wasn't intended to be an investment. It was introduced as a payment method that attempted to remove the need to have regulatory agencies or third parties involved in transactions.

It became popular with investors once it was noted that there was the potential for gains. Investors poured into the new asset space, creating demand that the cryptocurrency's designers may not have anticipated. For investors, a halving represents a reduction in the new coin supply, but it also offers the promise of an increase in investment value if the event's effects remain the same. But this places Bitcoin investing into the realm of speculation because those invested in the cryptocurrency are hoping for gains.

Mining
Miners are the people, groups, or businesses that focus on mining for its profitability. When new Bitcoins are awarded, the miner(s) that receive the reward have been making substantial gains in the past. As Bitcoin's price fluctuated over the years, it remained a lucrative endeavor—if it hadn't, the large mining businesses wouldn't have continued operating.

However, a halving cuts mining rewards, so the endeavor becomes less profitable with each halving if prices remain the same or drop. The large-scale mining facilities needed to remain competitive require enormous amounts of money and energy. The equipment and facilities need maintenance and people to conduct it. They also need to upgrade their mining capacity to maintain their position in the industry.

For instance, Marathon Digital Holdings, one of the world's largest mining firms, increased its Bitcoin holdings to 16,930 and its fleet of Bitcoin miners to 231,000 in February 2024. This brings the firm's hash rate to 28.7 trillion hashes per second (about 5% of the network's total hash rate as of April 19, 2024).
5
6

The increase in production capacity and holdings was likely due to anticipations of the April 2024 halving and the amount of hashing power required to remain competitive while having the liquidity necessary to finance its operations.

For smaller miners, a decrease in the reward means lower chances. Miners who are part of a mining pool will likely experience smaller rewards, even if prices increase—the reward is being cut in half, but Bitcoin's price is not likely to double to maintain current profitability unless there is a drastic market event.

Consumers
Consumers and retail Bitcoin users might be affected by a halving in the value of the Bitcoin they hold. Those who buy Bitcoin for making purchases will generally only be affected by price fluctuations, which may or may not remain similar to those before the halving occurred.

For those using Bitcoin for remittances, a halving means the same thing as it does for shoppers. The value of their remittances will depend on Bitcoin's market price after the halving event.
What Happens When Bitcoin Halves?
The term "halving" as it relates to Bitcoin concerns how many tokens are rewarded. This acts as a way to simulate diminishing returns, theoretically intended to raise demand.

Why Are the Halvings Occurring Less Than Every 4 Years?
The Bitcoin mining algorithm is set with a target of finding new blocks once every 10 minutes.
7
 Some blocks take more than 10 minutes; some take less. This can decrease or increase the amount of time it takes to reach the next halving goal. For example, if blocks consecutively average 9.66 minutes to mine, it would take about 1,409 days to mine the 210,000 blocks required (four years is 1461 days, including one day for a leap year).

What Happens When There Are No More Bitcoins Left?
It is often thought that in 2140, the last bitcoin will be mined. However, if the reward is halved every 210,000 blocks, it will get smaller and smaller until one satoshi is the reward and the total amount circulating equals 21 million. One satoshi is 0.00000001 bitcoin—it is the lowest denomination of Bitcoin and cannot be halved.

The Bottom Line
A Bitcoin halving cuts the rate at which new Bitcoins are released into circulation in half. The rewards system is expected to continue until 2140 when the proposed limit of 21 million bitcoin is theoretically reached.

In 2009, the reward for each block in the chain mined was 50 bitcoins. After the first halving, it was 25, and then 12.5, and then 6.25 bitcoins on May 11, 2020. The reward was reduced to 3.125 when the latest halving occurred on April 19, 2024.

Bitcoin halving has major implications for its network. For miners, the halving event may result in consolidation in their ranks as individual miners and small outfits drop out of the mining ecosystem or are taken over by larger players.

The comments, opinions, and analyses expressed on Investopedia are for informational purposes online. Read our warranty and liability disclaimer for more info. As of the date this article was written, the author does not own Bitcoin.
SPONSORED
Personalized Advice When You Need It
Put your financial goals in motion with an expert-built portfolio and receive ongoing advice and annual check-ins to ensure your plan is on track. With J.P. Morgan Personal Advisors, you can get a professionally designed investment portfolio that meets your needs and automatically rebalances as the market shifts. Fill out this short form to get started.'''

test_splited = split_large_text(doc, 30)

docs_embeddings = model.encode(test_splited, convert_to_tensor=True)

query = "tell me about effect of bitcoin halving in 2024"
query_embedding = model.encode(query, convert_to_tensor=True)

hits = semantic_search(query_embedding, docs_embeddings, top_k=10)
needed_docs = []
for hit in hits[0]:
    # print(test_splited[hit['corpus_id']], "(Score: %.4f)" % hit['score'])
    needed_docs.append(test_splited[hit['corpus_id']])

corpus = '\n'.join(needed_docs)

response = gen_model.generate_content(query + '\n' + corpus)
print(response.text)


**Bitcoin Halving in 2024**

The next Bitcoin halving is expected to occur on April 19, 2024, when the block reward will be reduced from 6.25 bitcoins to 3.125 bitcoins.

**Effect on Bitcoin Supply**

Halving reduces the rate at which new bitcoins are introduced into the market. This decreased supply can theoretically drive up demand and increase the price of Bitcoin.

**Historical Impact of Halvings**

Previous halvings have historically been associated with significant price increases for Bitcoin. However, it's important to note that correlation does not imply causation, and multiple factors can influence Bitcoin's price.

**Potential Impact on the Market**

**Consumers:**

* Those who hold Bitcoin may see an increase in the value of their holdings if the price rises.
* Individuals who use Bitcoin for remittances may face higher transaction costs as the reward decrease reduces miners' operating margins.

**Miners:**

* Miners will receive a smaller reward for their efforts, which cou

In [14]:
response = gen_model.generate_content(query)
print(response.text)

**Impact of Bitcoin Halving in 2024**

The Bitcoin halving is a scheduled event that occurs every four years, whereby the block reward for mining a Bitcoin block is reduced by half. The next halving is projected to occur in May 2024.

**Effects on Supply and Demand:**

* **Reduced Supply:** The halving will decrease the issuance rate of new Bitcoins by 50%, from the current reward of 6.25 BTC to 3.125 BTC per block. This will tighten the overall supply of Bitcoins in the market.
* **Increased Demand:** The halving is expected to increase demand for Bitcoin due to its limited supply. As the supply becomes scarcer, investors and traders may be more inclined to purchase and hold Bitcoin for its perceived scarcity value.

**Impact on Price:**

* **Potential Price Increase:** Historically, Bitcoin halvings have been associated with price surges. The scarcity created by the reduction in supply tends to drive up demand and prices.
* **Volatility:** The halving can lead to increased market vol